#### Импортируем библиотеки

In [20]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pytest
import ipytest

#### Фиксируем случайное состояние для воспроизводимости результатов

In [75]:
np.random.seed(40)

#### Генерируем данные для 3-х датасетов (площадь, квартиры, количество комнат, возраст дома и цена)

In [76]:
# Генерируем данные для тренировочного датасета
n_samples = 1000
area_train = np.random.normal(50, 20, n_samples)  # площадь квартиры
rooms_train = np.random.randint(1, 7, n_samples)  # количество комнат
age_train = np.random.randint(1, 50, n_samples)  # возраст дома
price_train = 500000 + 70000 * area_train + 100000 * rooms_train - 20000 * age_train + np.random.normal(0, 10000,
                                                                                                        n_samples)
# Генерируем данные для тестового датасета без изменений
area_test = np.random.normal(50, 20, n_samples)
rooms_test = np.random.randint(1, 7, n_samples)
age_test = np.random.randint(1, 50, n_samples)
price_test = 500000 + 70000 * area_test + 100000 * rooms_test - 20000 * age_test + np.random.normal(0, 10000, n_samples)

# Генерируем данные для тестового датасета с добавлением шума
price_test_noisy = price_test + np.random.normal(0, 1000000, n_samples)

#### Преобразуем в DataFrame для удобства

In [77]:
train_data = pd.DataFrame({'area': area_train, 'rooms': rooms_train, 'age': age_train, 'price': price_train})
test_data = pd.DataFrame({'area': area_test, 'rooms': rooms_test, 'age': age_test, 'price': price_test})
test_data_noisy = pd.DataFrame({'area': area_test, 'rooms': rooms_test, 'age': age_test, 'price': price_test_noisy})

#### Обучаем модель на нашем тренировочном датасете (будем предсказывать цену квартиры на основе количества комнат, площади и возраста дома)

In [78]:
model = LinearRegression()
model.fit(train_data[['area', 'rooms', 'age']], train_data['price'])

LinearRegression()

#### Функция для вычисления R-квадрата

In [79]:
def r2_calc(model, data):
    x_test = data[['area', 'rooms', 'age']]
    y_test = data['price']
    y_pred = model.predict(x_test)
    r2 = r2_score(y_test, y_pred)
    return r2

#### Функция для вычисления MAE (Средняя абсолютная ошибка)

In [80]:
def mae_calc(model, data):
    x_test = data[['area', 'rooms', 'age']]
    y_test = data['price']
    y_pred = model.predict(x_test)
    mae = mean_absolute_error(y_test, y_pred)
    return mae

#### Вычисляем R2 и MAE для каждого датасета, используя нашу обученную модель

In [81]:
r2_train = r2_calc(model, train_data)
r2_test = r2_calc(model, test_data)
r2_test_noisy = r2_calc(model, test_data_noisy)
mae_train = mae_calc(model, train_data)
mae_test = mae_calc(model, test_data)
mae_test_noisy = mae_calc(model, test_data_noisy)

#### Тестируем (Используем метрики R2 и MAE, значения которых бы нас удовлетворяли для данного датасета. Также сравниваем эти метрики между разными датасетами (самые лучшие метрики у датасета, на основе которого происходило обучение модели, чуть хуже у тестового датасета и еще хуже у датасета с шумом))

In [84]:
%%ipytest
def test_r2():
    assert ((r2_train > 0.99) & (r2_test > 0.95) & (r2_test_noisy > 0.5))


def test_mae():
    assert ((mae_train < 100000) & (mae_test < 100000) & (mae_test_noisy < 1000000))


def test_r2_comparison():
    assert r2_test_noisy < r2_test < r2_train


def test_mae_comparison():
    assert mae_train < mae_test < mae_test_noisy

....                                                                                         [100%]
4 passed in 0.02s


#### Значения метрик для наглядности

In [85]:
print(f'R2 на тренировочном датасете: {r2_train}')
print(f'R2 на тестовом датасете: {r2_test}')
print(f'R2 на датасете с шумом: {r2_test_noisy}')
print(f'MAE на тренировочном датасете: {mae_train}')
print(f'MAE на тестовом датасете: {mae_test}')
print(f'MAE на датасете с шумом: {mae_test_noisy}')

R2 на тренировочном датасете: 0.9999525026320507
R2 на тестовом датасете: 0.9999503908213849
R2 на датасете с шумом: 0.6740807984001558
MAE на тренировочном датасете: 7913.559369057989
MAE на тестовом датасете: 8106.964156870771
MAE на датасете с шумом: 787207.8459803206


#### ВЫВОД

Наиболее точно модель предсказывает цену квартиры на тренировочном датасете (что и логично) в среднем ошибка 7.900 рублей при средней стоимости одной квартиры 4 млн рублей. Наименее точно модель работает на тестовом датасете, там средняя ошибка составляет ~8.100 рублей. Наихудшее предсказание на датасете с шумом, при том, что чем больше мы задаем шум, тем хуже будут результаты обученной модели. В нашем случае модель на данном датасете в среднем ошибается на 787.000 рублей или ~ 20%, что делает использование модели на данном датасете не такой практичной.